In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
sys.argv = [sys.argv[0]]

In [47]:
from alphaledger.universe import Universe
from alphaledger.config import settings

universe = Universe("indices/sp100", start_year=2021)

# universe = Universe(settings.universe_name)
# filings_df = universe.get_filings()

# numeric_df, text_df = process_filing_urls(filings_df)

INFO:alphaledger.universe:Loading universe definition from: /Users/forrest.murray/Documents/alphaledger/universes/indices/sp100.yaml


INFO:alphaledger.universe:Filings metadata file not found at expected path: /Users/forrest.murray/Documents/alphaledger/output/sec_filings/S_P_100_Index.delta. Run collect_filings() to create it.
INFO:alphaledger.universe:Initialized Universe 'S_P_100_Index' from /Users/forrest.murray/Documents/alphaledger/universes/indices/sp100.yaml with 96 securities. Filings metadata status: Not Found.


In [48]:
universe.securities

{'AAPL': Security(ticker='AAPL', name='Apple Inc.', exchange='NASDAQ', sector='Information Technology', industry='Consumer Electronics', currency='USD', country='US', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'ACN': Security(ticker='ACN', name='Accenture plc', exchange='NYSE', sector='Information Technology', industry='Information Technology Services', currency='USD', country='IE', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'ADBE': Security(ticker='ADBE', name='Adobe Inc.', exchange='NASDAQ', sector='Information Technology', industry='Software—Infrastructure', currency='USD', country='US', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'AMD': Security(ticker='AMD', name='Advanced Micro Devices, Inc.', exchange='NASDAQ', sector='Information Technology', industry='Semiconductors', currency='USD', country='US', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'AMAT': Security(ticker=

In [49]:
universe.year_range

YearRange(start=2021, end=2025)

In [50]:
# u = Universe("sectors/cloud_computing")   
filings = universe.collect_filings()

INFO:alphaledger.universe:Ensuring filings metadata file is available and complete for 'S_P_100_Index' ([2021-2025])...
INFO:alphaledger.universe:Found 480 missing filing metadata combinations to fetch for ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMAT', 'AVGO', 'CRM', 'CSCO', 'GOOGL', 'IBM', 'INTC', 'INTU', 'META', 'MSFT', 'NOW', 'NVDA', 'ORCL', 'PLTR', 'QCOM', 'TXN', 'ABBV', 'ABT', 'AMGN', 'BMY', 'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'TMO', 'UNH', 'AIG', 'AXP', 'BAC', 'BK', 'BLK', 'BRK.B', 'C', 'COF', 'GS', 'JPM', 'MA', 'MET', 'MS', 'PYPL', 'SCHW', 'USB', 'V', 'WFC', 'AMZN', 'BKNG', 'CL', 'COST', 'DIS', 'GM', 'HD', 'LOW', 'MDLZ', 'MCD', 'MO', 'NKE', 'PG', 'SBUX', 'TGT', 'TSLA', 'WMT', 'PEP', 'CHTR', 'CMCSA', 'NFLX', 'TMUS', 'T', 'VZ', 'BA', 'CAT', 'COP', 'CVX', 'DE', 'DUK', 'EMR', 'FDX', 'GD', 'GE', 'HON', 'LMT', 'NEE', 'RTX', 'SO', 'SPG', 'UNP', 'UPS', 'XOM'] years [2021, 2022, 2023, 2024, 2025].
INFO:alphaledger.universe:Attempting to fetch metadata for 480 missing

In [51]:
filings.collect()

ticker,cik,form,accessionNumber,filingDate,filing_date_dt,filing_year,reportDate,report_date_dt,primaryDocument,primaryDocDescription,documents_url,xbrl_instance_url,processed_datetime
str,str,str,str,str,date,i64,str,date,str,str,str,str,datetime[μs]
"""AAPL""","""0000320193""","""10-K""","""0000320193-21-000105""","""2021-10-29""",2021-10-29,2021,"""2021-09-25""",2021-09-25,"""aapl-20210925.htm""","""10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:13:41.764073
"""ABT""","""0000001800""","""10-K""","""0001628280-24-005348""","""2024-02-16""",2024-02-16,2024,"""2023-12-31""",2023-12-31,"""abt-20231231.htm""","""10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:13:45.052688
"""AIG""","""0000005272""","""10-K""","""0001104659-22-024701""","""2022-02-17""",2022-02-17,2022,"""2021-12-31""",2021-12-31,"""aig-20211231.htm""","""FORM 10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:13:50.822856
"""AMD""","""0000002488""","""10-K""","""0000002488-25-000012""","""2025-02-05""",2025-02-05,2025,"""2024-12-28""",2024-12-28,"""amd-20241228.htm""","""10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:13:54.310470
"""AVGO""","""0001730168""","""10-K""","""0001730168-22-000118""","""2022-12-16""",2022-12-16,2022,"""2022-10-30""",2022-10-30,"""avgo-20221030.htm""","""10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:13:56.813316
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TMO""","""0000097745""","""10-K""","""0000097745-25-000010""","""2025-02-20""",2025-02-20,2025,"""2024-12-31""",2024-12-31,"""tmo-20241231.htm""","""10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:18:00.424229
"""TXN""","""0000097476""","""10-K""","""0000097476-22-000009""","""2022-02-04""",2022-02-04,2022,"""2021-12-31""",2021-12-31,"""txn-20211231.htm""","""10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:18:05.031254
"""UNP""","""0000100885""","""10-K""","""0001437749-24-003599""","""2024-02-09""",2024-02-09,2024,"""2023-12-31""",2023-12-31,"""unp20231231_10k.htm""","""FORM 10-K""","""https://www.sec.gov/Archives/e…","""https://www.sec.gov/Archives/e…",2025-05-12 15:18:08.926573


In [52]:
univ_numeric = universe.get_numeric_facts()

INFO:alphaledger.universe:Data file not found: No numeric facts found for universe 'S_P_100_Index' at /Users/forrest.murray/Documents/alphaledger/output/numeric_facts/S_P_100_Index.delta. Processing required.
INFO:alphaledger.universe:Processing numeric facts for universe 'S_P_100_Index' using direct method...
<string>:32: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
INFO:alphaledger.universe:Found 455 filings for universe 'S_P_100_Index'. Processing XBRL using direct method...
INFO:alphaledger.process_xbrl:[process_filing_urls_direct] Starting direct fact extraction.
ERROR:alphaledger.sec:Error parsing XBRL for PLTR (Filing Date: N/A): undefined entity: line 15, column 59
Traceback (most recent call last):
  File "/Users/forrest.murray/Documents/alphaledger/src/alphaledger/sec.py", line 924, in parse_filing

In [54]:
univ_numeric.collect()

concept_name,concept_namespace,fact_type,period_instant,period_start,period_end,context_id,context_entity,context_scenario,filing_date,report_date,fact_value,unit,metadata,ticker
str,str,str,date,date,date,str,str,str,date,date,f64,str,struct[2],str
"""RevenueRemainingPerformanceObl…","""elts""","""NumericFact""",2021-09-25,null,null,null,null,null,2021-10-29,2021-09-25,0.64,"""xbrli:pure""","{2,null}","""AAPL"""
"""RevenueRemainingPerformanceObl…","""elts""","""NumericFact""",2021-09-25,null,null,null,null,null,2021-10-29,2021-09-25,0.26,"""xbrli:pure""","{2,null}","""AAPL"""
"""RevenueRemainingPerformanceObl…","""elts""","""NumericFact""",2021-09-25,null,null,null,null,null,2021-10-29,2021-09-25,0.08,"""xbrli:pure""","{2,null}","""AAPL"""
"""RevenueRemainingPerformanceObl…","""elts""","""NumericFact""",2021-09-25,null,null,null,null,null,2021-10-29,2021-09-25,0.02,"""xbrli:pure""","{2,null}","""AAPL"""
"""EntityPublicFloat""","""2021""","""NumericFact""",2021-03-26,null,null,null,null,null,2021-10-29,2021-09-25,2.0214e12,"""iso4217:USD""","{-6,null}","""AAPL"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ContractWithCustomerLiability""","""elts""","""NumericFact""",2021-12-31,null,null,null,null,null,2022-02-24,2021-12-31,3.3000e9,"""iso4217:USD""","{-8,null}","""PFE"""
"""ContractWithCustomerLiability""","""elts""","""NumericFact""",2020-12-31,null,null,null,null,null,2022-02-24,2021-12-31,9.57e8,"""iso4217:USD""","{-6,null}","""PFE"""
"""ContractWithCustomerLiabilityC…","""elts""","""NumericFact""",2021-12-31,null,null,null,null,null,2022-02-24,2021-12-31,3.0000e9,"""iso4217:USD""","{-8,null}","""PFE"""


In [61]:
TAGS = {
    # — income statement —
    "Revenues":                                   "revenue_total",
    "SalesRevenueNet":                            "revenue_total",
    "RevenueFromContractWithCustomerExcludingAssessedTax": "revenue_total",
    "NetSales":                                   "revenue_total",
    "SalesRevenueGoodsNet":                       "revenue_goods",
    "SalesRevenueServicesNet":                    "revenue_services",
    "InterestAndDividendIncomeOperating":         "revenue_interest_dividends",
    "TotalRevenueNetOfInterestExpense":           "revenue_net_of_interest_expense",

    "CostOfGoodsAndServicesSold":                 "cost_of_goods_and_services_sold",
    "CostOfRevenue":                              "cost_of_revenue",
    "CostOfGoodsSold":                            "cost_of_goods_sold",
    "CostOfGoodsAndServiceExcludingDepreciationDepletionAndAmortization":
                                                         "cogs_excl_d_and_a",

    "GrossProfit":                                "gross_profit",
    "OperatingIncomeLoss":                        "operating_income",
    "OperatingIncome":                            "operating_income",
    "NetIncomeLoss":                              "net_income",
    "ProfitLoss":                                 "net_income",

    # — earnings per share —
    "EarningsPerShareBasic":                      "eps_basic_total",
    "IncomeLossFromContinuingOperationsPerBasicShare":
                                                         "eps_basic_cont_ops",
    "EarningsPerShareDiluted":                    "eps_diluted_total",
    "IncomeLossFromContinuingOperationsPerDilutedShare":
                                                         "eps_diluted_cont_ops",

    # — operating expenses —
    "ResearchAndDevelopmentExpense":              "research_and_development_expense",
    "ResearchAndDevelopment":                     "research_and_development_expense",
    "ResearchAndDevelopmentExpenseCredit":        "research_and_development_expense",
    "SellingGeneralAndAdministrativeExpense":     "selling_general_admin_expense",
    "GeneralAndAdministrativeExpense":            "general_admin_expense",

    # — taxes —
    "IncomeTaxExpenseBenefit":                    "income_tax_expense",
    "CurrentIncomeTaxExpenseBenefit":             "current_income_tax_expense",
    # alternate tax tag some filers use
    "ProvisionForIncomeTaxes":                       "income_tax_expense",

    # — cash-flow statement —
    "NetCashProvidedByUsedInOperatingActivities": "cash_flow_from_operations",
    "NetCashProvidedByUsedInInvestingActivities": "cash_flow_from_investing",
    "NetCashProvidedByUsedInFinancingActivities": "cash_flow_from_financing",

    "CapitalExpenditures":                        "capital_expenditures",
    "PaymentsToAcquirePropertyPlantAndEquipment": "capital_expenditures",
    "PaymentsToAcquireOtherPropertyPlantAndEquipment":
                                                         "capital_expenditures",

    "RepurchaseOfCommonStock":                    "share_repurchases",
    "PaymentsForRepurchaseOfCommonStock":         "share_repurchases",
    "AcceleratedShareRepurchasesSettlementPaymentOrReceipt":
                                                         "share_repurchases",
    # share repurchases alt tag
    "StockRepurchasedAndRetiredDuringPeriodValue":   "share_repurchases",

    # — balance sheet (assets, liabilities, equity) —
    "CashAndCashEquivalentsAtCarryingValue":      "cash_and_cash_equivalents",
    "CashCashEquivalentsAndShortTermInvestments": "cash_equiv_and_short_term_investments",

    "AccountsReceivableNet":                      "accounts_receivable_net",
    "AccountsAndNotesReceivableNet":              "accounts_and_notes_receivable_net",
    "AccountsReceivableNetCurrent":               "accounts_receivable_current",

    "InventoryNet":                               "inventory_net",
    "InventoryFinishedGoodsAndWorkInProcess":     "inventory_finished_goods_wip",

    "AccountsPayableCurrent":                     "accounts_payable_current",
    "AccountsPayableTradeCurrent":                "accounts_payable_trade_current",

    "Assets":                                     "assets_total",
    "AssetsNet":                                  "assets_total_net",
    "AssetsCurrent":                              "assets_current",

    "Liabilities":                                "liabilities_total",
    "LiabilitiesCurrent":                         "liabilities_current",

    "StockholdersEquity":                         "stockholders_equity",

    # — debt (current v. long-term v. total) —
    "LongTermDebtAndCapitalLeaseObligations":     "long_term_debt_and_capital_leases",
    "LongTermDebt":                               "long_term_debt",
    "DebtCurrent":                                "debt_current",
    "CommercialPaper":                            "commercial_paper",
    "ShortTermBorrowings":                        "short_term_borrowings",
    "CurrentPortionOfLongTermDebt":               "current_portion_long_term_debt",
    "LongTermDebtCurrent":                        "current_portion_long_term_debt_alt",
    "FinanceLeaseLiabilityCurrent":               "finance_lease_liability_current",
    "FinanceLeaseLiabilityNoncurrent":            "finance_lease_liability_non_current",
    "Debt":                                       "debt_total",

    # — intangible assets —
    "Goodwill":                                   "goodwill",
}

In [65]:
import polars as pl
print("DataFrame columns:", univ_numeric.columns)

numeric_df = univ_numeric.collect()

unique_concepts = (numeric_df
    .select("concept_name")
    .unique()
    .to_dict()["concept_name"])

tag_keys = set(TAGS.keys())

found_tags = tag_keys.intersection(set(unique_concepts))

print(f"Total number of numeric facts: {len(numeric_df)}")
print(f"Number of unique tags in TAGS: {len(tag_keys)}")
print(f"Number of tags from TAGS found in data: {len(found_tags)}")

coverage = len(found_tags) / len(tag_keys) * 100
print(f"\nCoverage: {coverage:.1f}% of TAGS are present in the data")

missing_tags = tag_keys - set(unique_concepts)
print(f"\nNumber of missing tags: {len(missing_tags)}")
if missing_tags:
    print("Missing tags:")
    for tag in sorted(missing_tags):
        print(f"  - {tag}")

DataFrame columns: ['concept_name', 'concept_namespace', 'fact_type', 'period_instant', 'period_start', 'period_end', 'context_id', 'context_entity', 'context_scenario', 'filing_date', 'report_date', 'fact_value', 'unit', 'metadata', 'ticker']
Total number of numeric facts: 1168177
Number of unique tags in TAGS: 65
Number of tags from TAGS found in data: 53

Coverage: 81.5% of TAGS are present in the data

Number of missing tags: 12
Missing tags:
  - CostOfGoodsSold
  - CurrentPortionOfLongTermDebt
  - NetSales
  - OperatingIncome
  - ProvisionForIncomeTaxes
  - RepurchaseOfCommonStock
  - ResearchAndDevelopment
  - ResearchAndDevelopmentExpenseCredit
  - SalesRevenueGoodsNet
  - SalesRevenueNet
  - SalesRevenueServicesNet
  - TotalRevenueNetOfInterestExpense


/var/folders/rh/d1q87pv57wqfxv9nymwnbjf00000gp/T/ipykernel_4458/3160829745.py:2: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print("DataFrame columns:", univ_numeric.columns)


In [58]:
numeric_df.unique("concept_name")

concept_name,concept_namespace,fact_type,period_instant,period_start,period_end,context_id,context_entity,context_scenario,filing_date,report_date,fact_value,unit,metadata,ticker
str,str,str,date,date,date,str,str,str,date,date,f64,str,struct[2],str
"""DeferredTaxLiabilitiesLeasesRi…","""000073271224000010""","""NumericFact""",2023-12-31,null,null,null,null,null,2024-02-09,2023-12-31,5.2000e9,"""iso4217:USD""","{-6,null}","""VZ"""
"""DefinedBenefitPlanBenefitOblig…","""000031361625000043""","""NumericFact""",null,2024-01-01,2024-12-31,null,null,null,2025-02-20,2024-12-31,4.6e7,"""iso4217:USD""","{-6,null}","""DHR"""
"""PercentageOfRent""","""000006390823000012""","""NumericFact""",null,2022-01-01,2022-12-31,null,null,null,2023-02-24,2022-12-31,3.96e7,"""iso4217:USD""","{-5,null}","""MCD"""
"""DefinedBenefitPlanAssumptionsU…","""000001292724000010""","""NumericFact""",null,2023-01-01,2023-12-31,null,null,null,2024-01-31,2023-12-31,0.05,"""xbrli:pure""","{4,null}","""BA"""
"""DeferredRevenueAssociatedWithF…","""000085887721000013""","""NumericFact""",2021-07-31,null,null,null,null,null,2021-09-09,2021-07-31,-1.6e7,"""iso4217:USD""","{-6,null}","""CSCO"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""OperatingLeaseLeaseIncomeLease…","""elts""","""NumericFact""",null,2021-11-01,2022-10-30,null,null,null,2022-12-15,2022-10-30,1.3180e9,"""iso4217:USD""","{-6,null}","""DE"""
"""NotesAndLoansReceivableGrossNo…","""elts""","""NumericFact""",2020-12-31,null,null,null,null,null,2021-02-12,2020-12-31,1.8050e9,"""iso4217:USD""","{-6,null}","""GE"""
"""MergerCommitmentSettlementDueI…","""000128369921000039""","""NumericFact""",2020-12-31,null,null,null,null,null,2021-02-23,2020-12-31,3.7e7,"""iso4217:USD""","{-6,null}","""TMUS"""


In [60]:
asset_concepts = (numeric_df
    .filter(pl.col("concept_name").str.contains("Asset"))
    .group_by("concept_name")
    .count()
    .sort("count", descending=True))

print("Number of unique concept names containing 'Asset':", len(asset_concepts))
print("\nAll concept names containing 'Asset' (sorted by frequency):")
print(asset_concepts)

Number of unique concept names containing 'Asset': 2109

All concept names containing 'Asset' (sorted by frequency):
shape: (2_109, 2)
┌─────────────────────────────────┬───────┐
│ concept_name                    ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ DefinedBenefitPlanFairValueOfP… ┆ 31219 │
│ Assets                          ┆ 6798  │
│ DefinedBenefitPlanFairValueOfP… ┆ 5168  │
│ DerivativeFairValueOfDerivativ… ┆ 4343  │
│ AssetsFairValueDisclosure       ┆ 3639  │
│ …                               ┆ …     │
│ BusinessCombinationRecognizedI… ┆ 1     │
│ AssetAcquisitionCashOutflowNet… ┆ 1     │
│ DefinedBenefitPlanProjectedInc… ┆ 1     │
│ AssetAcquisitionRecognizedIden… ┆ 1     │
│ CryptoAssetNumberOfUnits        ┆ 1     │
└─────────────────────────────────┴───────┘


/var/folders/rh/d1q87pv57wqfxv9nymwnbjf00000gp/T/ipykernel_4458/2648266635.py:4: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  .count()


In [73]:
# Get unique filing_date + ticker combinations
unique_filings = (univ_numeric
    .select(["filing_date", "ticker"])
    .unique()
    .collect())
total_filings = len(unique_filings)

print(f"Total number of unique filing_date + ticker combinations: {total_filings}")

# For each concept, count how many filings it appears in
concept_coverage = (univ_numeric
    .with_columns(
        pl.concat_str([pl.col("filing_date"), pl.col("ticker")], separator="_").alias("filing_key")
    )
    .group_by("concept_name")
    .agg(
        pl.col("filing_key").n_unique().alias("num_filings")
    )
    .sort("num_filings", descending=True))

# Find concepts that appear in all filings
complete_coverage = concept_coverage.filter(pl.col("num_filings") == total_filings)

print(f"\nNumber of concepts present in all filings: {len(complete_coverage.collect())}")
print("\nConcepts present in all filings:")
print(complete_coverage.collect())

# Show concepts with high coverage (e.g., >90% of filings)
high_coverage = concept_coverage.filter(pl.col("num_filings") >= total_filings * 0.9)
print(f"\nNumber of concepts present in >90% of filings: {len(high_coverage.collect())}")
print("\nTop concepts by coverage:")
print(high_coverage.head(10).collect())

Total number of unique filing_date + ticker combinations: 453

Number of concepts present in all filings: 2

Concepts present in all filings:
shape: (2, 2)
┌─────────────────────────────────┬─────────────┐
│ concept_name                    ┆ num_filings │
│ ---                             ┆ ---         │
│ str                             ┆ u32         │
╞═════════════════════════════════╪═════════════╡
│ EntityPublicFloat               ┆ 453         │
│ EntityCommonStockSharesOutstan… ┆ 453         │
└─────────────────────────────────┴─────────────┘

Number of concepts present in >90% of filings: 24

Top concepts by coverage:
shape: (10, 2)
┌─────────────────────────────────┬─────────────┐
│ concept_name                    ┆ num_filings │
│ ---                             ┆ ---         │
│ str                             ┆ u32         │
╞═════════════════════════════════╪═════════════╡
│ EntityCommonStockSharesOutstan… ┆ 453         │
│ EntityPublicFloat               ┆ 453         │
│